# Converting to sens,spec table

In [ ]:
%load_ext autoreload
%autoreload 2

import globvars as gv
import utils
import utils_tSNE as utsne
import utils_preproc as upre

import os
import sys
import mne
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
import multiprocessing as mpr
import matplotlib as mpl
import time
import gc;
import scipy.signal as sig
import pandas as pd 

import numpy as np
import utils_postprocess as upost

data_dir = gv.data_dir


In [ ]:
#fname_full = os.path.join(data_dir, 'S02_off_hold_resample_notch_highpass_raw.fif')



In [ ]:
tdelta.total_seconds()

In [ ]:
gen_subj_info = gv.gen_subj_info

In [ ]:
rawnames =   ['S01_off_hold', 'S01_off_move'] + ['S01_on_hold', 'S01_on_move']
rawnames +=  ['S02_off_hold', 'S02_off_move'] + ['S02_on_hold', 'S02_on_move'] 
rawnames +=  ['S03_off_hold', 'S03_off_move']
rawnames +=  ['S04_off_hold', 'S04_off_move'] + ['S04_on_hold', 'S04_on_move'] 
rawnames +=  ['S05_off_hold', 'S05_off_move'] + ['S05_on_hold', 'S05_on_move'] 
rawnames +=  ['S07_off_hold', 'S07_off_move'] + ['S07_on_hold', 'S07_on_move'] 

rawnames = rawnames[::2] # we have joined them anyway

In [ ]:
rawnames =   ['S01_off_hold']
rawnames +=  ['S02_off_hold'] 
rawnames +=  ['S04_off_hold'] 
rawnames +=  ['S07_off_hold'] 

In [ ]:
rawnames =   ['S01_off_hold']
rawnames +=  ['S02_off_hold'] 
rawnames +=  ['S03_off_hold']
rawnames +=  ['S04_off_hold'] 
rawnames +=  ['S05_off_hold'] 
rawnames +=  ['S07_off_hold'] 

In [ ]:
rawnames =   ['S01_off_hold', 'S01_off_move'] + ['S01_on_hold']

In [ ]:
rawnames =  ['S01_off_hold']

In [ ]:
rawnames =  ['S99_off_hold', 'S99_off_move']

In [ ]:
# rawnames = ['S01_off_hold', 'S02_off_hold', 'S02_on_hold', 
#             'S03_off_hold', 'S04_off_hold', 'S04_on_hold',
#            'S05_off_hold', 'S05_on_hold', 'S07_off_hold'] #,'S01_on_hold']

In [ ]:
rawnames

In [ ]:
prefixes = ['all',
'allb_trem',    
'allb_beta',    
'allb_gamma',   
'allb_trembeta',
'allnoHFO',
'modLFP',
'modLFPnoHFO',
'modSrc',
'onlyTD',
'onlyFD',
'conH',
'onlyH',
'onlyBpcorr',
'onlyBpcorrNoHFO',
'onlyRbcorr',
'onlyCon',
'onlyConNoHFO',
'LFPrel']

In [ ]:
prefixes = ['all',
'allb_trem',    
'allb_beta',    
'allb_gamma',   
'modLFP',
'modSrc',
'modSrc_self',
'onlyMotorSrc',
'onlyRestSrc', 
'onlyCBSrc'    ]


In [ ]:
prefixes = ['all',
'allb_trem',    
'allb_beta',    
'allb_gamma',  
'allb_HFO',
'modLFP',
'modSrc',
'modSrc_self',
'LFPrel_noself',
'LFPrel_noself_onlyCon',            
'LFPrel_noself_onlyRbcorr',            
'LFPrel_noself_onlyBpcorr',                        
'onlyTD', 'onlyFD', 'allb_trembeta',
'onlyMotorSrc','onlyRestSrc','onlyCBSrc' ]

In [ ]:
rawnames = ['S01']

In [ ]:
prefixes = None

In [ ]:
#%debug
#label_types = ['allsep', 'trem_vs_all', 'trem_vs_quiet']
#label_types = ['allsep']

nraws_used_PCA = 2

set_explicit_nraws_used_PCA = 1
set_explicit_n_feats_PCA = 0
set_explicit_dim_PCA = 0



sources_type = 'parcel_aal'  # or ''

try:
    print(output_per_raw.keys() )
except (NameError,AttributeError) as e:
    output_per_raw = None
    
#subdir='SSS'
subdir =''
#%debug
r = upost.collectPerformanceInfo(rawnames,prefixes,                                                                                                   
                                          sources_type = sources_type, 
                                           printFilenames=1,                                 
                                                         ndays_before=20,
                                             subdir=subdir, old_file_format=1)
output_per_raw,Ximp_per_raw,gis_per_raw = r

import gc; gc.collect()

In [ ]:
output_per_raw

In [ ]:
output_per_raw['S01_off_hold'].keys()

In [ ]:
# '()' by default means grouping (functional)
import re
r = re.match('.*\+.+\((\d+)\)','  fdsfd+f (5454)')
print(r)
print( r.groups() )

In [ ]:
r.regs

In [ ]:
_S01_parcel_aal_grp10-0_modSrc_short_ML_nr1_7chs_nfeats612_pcadim268_skip32_wsz256__(merge_all_not_trem,trem_vs_quiet).npz

In [ ]:
Ximp_per_raw

In [ ]:
output_per_raw

In [ ]:
# Collect feature numbers
feat_nums_perraw0 = {}
feat_nums_perraw = {}
feat_nums_red_perraw = {}
for rn in output_per_raw:
    #feat_nums_perprefix = {}
    feat_nums0 = {}
    feat_nums_per_prefix = {}
    feat_nums_red_per_prefix = {}
    for prefix,pg in output_per_raw[rn].items():        
        feat_nums_red_pgs = {}
        feat_nums_pgs = {}
        for g,pitset in pg.items():
            if g == 'feature_names_filtered':
                continue            
            for it_set,lda_output in pitset.items():                
                if lda_output is None:
                    continue
                lda_anver = lda_output['lda_analysis_versions']
                nfeats = len( lda_anver['all_present_features']['CV_aver']['ldaobj'].scalings_ )
                siXGB = lda_output.get('strong_inds_XGB',None )
                if siXGB is not None:
                    nfeats_red_XGB = len(  siXGB )
                else:
                    nfeats_red_XGB = -1
                #X = lda_output['transformed_imputed_CV']                
                #nfeats = len( lda_output['MI_per_Feati'].feature_importances_ )
                #if 'XGBobj' in lda_output:
                #    nfeats = len( lda_output['XGBobj'].feature_importances_ )
                #else:
                #print(rn,prefix,g,it_set,X.shape)
                #n = feat_nums.get(prefix,-1)
#                 if n >= 0:
#                     assert nfeats == n, (nfeats,n)
#                 else:
#                     nfeats = -1
                    
                feat_nums0[prefix] = nfeats
                feat_nums_pgs[(g,it_set)] = nfeats
                feat_nums_red_pgs[(g,it_set)] = nfeats_red_XGB
        feat_nums_red_per_prefix[prefix] = feat_nums_red_pgs
        feat_nums_per_prefix[prefix] = feat_nums_pgs
        #feat_nums_perprefix[prefix ] = feat_nums
    feat_nums_perraw0[rn] = feat_nums0
    feat_nums_perraw[rn] = feat_nums_per_prefix
    feat_nums_red_perraw[rn] = feat_nums_red_per_prefix

#feat_nums_perraw
   

In [ ]:
     

# prefix_labels_perraw = {}
# for rawn in output_per_raw:
#     ks = list( feat_nums_perraw[rawn].keys() )
#     labs = ['{}:{}'.format(prefix,feat_nums_perraw[rawn][prefix] ) \
#             for prefix in ks ]
#     prefix_labels_perraw[rawn] = dict(zip(ks,labs))
# print( prefix_labels_perraw )

# prefix_labels_all_appeared = set()
# for rn,plp in prefix_labels_perraw.items():
#     prefix_labels_all_appeared.update(plp)
# # has wrong sort
# prefix_labels_all_appeared = list(sorted(prefix_labels_all_appearaed))



In [ ]:
feat_nums_perraw

In [ ]:
feat_nums_red_perraw

In [ ]:
lda_output.keys()

# Table of sens,spec

In [ ]:
prefixes

In [ ]:
to_show = [('allsep','merge_nothing','basic'), ('trem_vs_all','merge_all_not_trem','basic'),
        ('trem_vs_quiet','merge_all_not_trem','trem_vs_quiet')]

to_show = [('allsep','merge_nothing','basic'),
           ('trem_vs_quiet','merge_nothing','trem_vs_quiet'),
           ('trem_vs_all','merge_all_not_trem','basic'),
           ('medcond','merge_within_medcond','subj_medcond'),
          ('dataset','merge_within_task','subj_medcond_task') ]

use_CV_perf = True

perf_to_use_list = ['perfs_XGB', 'perfs_XGB_red', 'all_present_features',
                    'strongest_features_LDA_selMinFeatSet', 
                    'strongest_features_XGB_opinion', 
                    'best_PCA-derived_features_0.6']

#label_types = [tpl[0] for tpl in to_show]
#print(label_types)
show_F1 = False
show_numfeats = False

assert output_per_raw is not None
rname_crop = slice(0,-5) # keeping medcond info
rname_crop = slice(0,3) # keeping medcond info

import pandas as pd 
for perf_to_use in perf_to_use_list:
    red_mode = False
    if perf_to_use.endswith('_red'):
        red_mode = True
    #table =  [ [''] +  prefix_labels_perraw[rawnames[0] ]
    #table =  [ [''] +  [ dct.get(prefix,prefix) for dct in prefix_labels_perraw] ]
    table =  [ [''] +  prefixes ]
    was_valid = False
    for rn in output_per_raw:
        for lt, it_grp, it_set in to_show:
        #for lt in label_types:
            # raw and label type (grouping+int_types) name goes here
            # this will be a row name
            row_name = '{}_{}'.format(rn[rname_crop], lt)
            table_row = [row_name ]
            for prefix in prefixes:
                #sens,spec = res[rn].get(pref, (np.nan, np.nan))
                r = output_per_raw[rn].get(prefix, None)     
                if (r is not None) and (it_grp not in r or it_set not in r[it_grp]):
                    r = None
                
                perfs = None
                if r is None:
                    print('Warning :',rn,prefix)
                else:
                    lda_output = r[it_grp][it_set]  
                    if lda_output is not None:
                        if perf_to_use in ['perfs_XGB','perfs_XGB_red']:
                            if 'perfs_XGB' in lda_output and lda_output['perfs_XGB'] is not None:
                                perfs_XGB = lda_output['perfs_XGB']
                                if perf_to_use == 'perfs_XGB':
                                    ind = 0
                                elif perf_to_use == 'perfs_XGB_red':
                                    ind = -1
                                perfs_noCV = perfs_XGB[ind][-2]
                                perfs_CV = perfs_XGB[ind][-1]                                                                                                    
                        else:
                            lda_anver = lda_output['lda_analysis_versions']                        
                            anver_cur = lda_anver.get(perf_to_use,None)
                            if anver_cur is not None:
                                perfs_CV = anver_cur['CV']['CV_perfs']
                                perfs_CV = np.mean(np.array(perfs_CV), axis=0)
                                perfs_CV2 = anver_cur['CV_aver']['perfs']
                                perfs_noCV = anver_cur['fit_to_all_data']['perfs']              
                        
                        if use_CV_perf:
                            perfs = perfs_CV
                        else:
                            perfs = perfs_noCV
                        
                if perfs is None:
                    print('Warning :',rn,prefix,lt)
                    sens,spec,F1 = np.nan, np.nan, np.nan
                else:
                    sens,spec,F1 = perfs
                    was_valid = True
                #str_to_put = '{:.0f},{:.0f}'.format(100*sens,100*spec)
                if show_F1:
                    str_to_put =  '{:.0f},{:.0f},{:.0f}'.format(100*sens,100*spec,100*F1)  
                else:
                    str_to_put =  '{:.0f},{:.0f}'.format(100*sens,100*spec)  
                try:
                    if show_numfeats:
                        num = feat_nums_perraw[rn][prefix][(it_grp,it_set)]
                        num_red = feat_nums_red_perraw[rn][prefix][(it_grp,it_set)]
                        if red_mode:
                            s = ' :{}/{}'.format(num_red,num)
                        else:
                            s = ' :{}/{}'.format(num,num)
                        str_to_put = str_to_put + s
                except KeyError as e:
                    print('AAAAA ',perf_to_use,rn,prefix,it_grp,it_set,e)
                table_row += [ str_to_put  ]
                print(rn,lt,prefix,str_to_put)

            table += [table_row]

        #rn,pref,lt

#         table = np.array(table)
        if not was_valid:
             print('All nan, skipping')
             continue

        table_fname = "pptable_{}_{}_nr{}_nprefixes{}_nlts{}_{}_CV{}.csv".\
        format(sources_type, subdir, len(rawnames), len(prefixes),
               len(to_show), perf_to_use, int(use_CV_perf) )
        table_fname_full = os.path.join(gv.dir_fig, table_fname)
        print(table_fname_full)
        
        pd.DataFrame(table).to_csv(table_fname_full)

In [ ]:
subdir

# Table of strongest feature

In [ ]:
#output_per_raw['S99_off_hold']['test'].keys()

In [ ]:
for perf_to_use in perf_to_use_list:
    table =  [ [''] +  prefix_labels_perraw[rawnames[0]]]
    was_valid = False
    for rn in output_per_raw:
        for lt, it_grp, it_set in to_show:
        #for lt in label_types:
            # raw and label type (grouping+int_types) name goes here
            # this will be a row name
            row_name = '{}_{}'.format(rn[rname_crop], lt)
            table_row = [row_name ]
            for prefix in prefixes:
                #sens,spec = res[rn].get(pref, (np.nan, np.nan))
                r = output_per_raw[rn].get(prefix, None)     
                if (r is not None) and (it_grp not in r or it_set not in r[it_grp]):
                    r = None
                
                perfs = None
                if r is None:
                    print('Warning :',rn,prefix)
                else:
                    lda_output = r[it_grp][it_set]  
                    if lda_output is not None:

In [ ]:
kord = list(res.keys())
#[k[:-5] for k in kord ]
table =  [ [''] +  prefixes]
for rn in kord:
    for lt in label_types:
        table_row = ['{}_{}'.format(rn[:-5], lt) ]
        for pref in prefixes:
            #sens,spec = res[rn].get(pref, (np.nan, np.nan))
            r = res[rn].get(pref, None)
            if r is None:
                print('Warning :',rn,pref)
                sfn = ''
            else:
                sfn = res_strongfeat[rn][pref][lt]
            print('{} {} {}\n  {}'.format(rn,lt,pref,sfn) )
            table_row += [ sfn  ]
        table += [table_row]

In [ ]:
table = np.array(table)

table_fname = "strongest_feat_table.csv"


pd.DataFrame(table).to_csv(table_fname)

In [ ]:
res_strongfeat.keys()

# Look at most important LFP

In [ ]:
chnames_LFP = ['LFPR01', 'LFPR12', 'LFPR23']
groupings_to_use = ['merge_nothing']
#rawnames_to_use = ['S99_off_move']
rawnames_to_use = rawnames
prefixes_to_use = ['modLFP', 'LFPrel']
prefixes_to_use = ['modLFP', 'LFPrel_noself']


In [ ]:
def selLFP_calcPerfDrops(chnames_LFP, rawnames_to_use = None, groupings_to_use = None,
                         prefixes_to_use = None, clf_type = 'XGB' ):
    perf_drops = {}
    # if all raws were processed together, they'll have same performances saved, no need to repeat

    #for k in output_per_raw:
    for ki,k in enumerate(rawnames_to_use):
        output_per_prefix = output_per_raw[k]
        #for prefix in output_per_prefix:
        for prefix in prefixes_to_use:
            output_per_grouping = output_per_prefix.get(prefix,None)
            if output_per_grouping is None:
                continue
            #for grouping in output_per_grouping:
            for grouping in groupings_to_use:
                output_per_int_types = output_per_grouping[grouping]
                for int_type in output_per_int_types:
                    output_cur = output_per_int_types[int_type]
                    #s = '{}:{}:{}:{}'.format(k,prefix,grouping,int_type)
                    s = '{}:{}:{}:{}'.format(ki,prefix,grouping,int_type)
                    if output_cur is None:
                        #print(k,prefix,grouping,int_type,'=None')
                        continue
                    lda_anver = output_cur['{}_analysis_versions'.format(clf_type)]
                    anver_full = lda_anver['all_present_features']
                    perfs_full = anver_full['CV_aver']['perfs']
                    perfs_full = np.array(perfs_full)
                    perfs_str_full = utsne.sprintfPerfs(perfs_full)
                    print('{}:: Full avCV perfs {}'.format(s,perfs_str_full))
                    perf_drops[s] = {}
                    for chn in chnames_LFP:
                        key = 'all_present_features_but_{}'.format(chn)
                        anver = lda_anver.get(key,None)
                        if anver is None:
                            #print(s,' fail')
                            break
                        perfs = np.mean(anver['CV']['CV_perfs'], axis=0)
                        perfs = np.array(perfs)

                        #print(perfs_full, perfs)
                        perfs_str = utsne.sprintfPerfs(perfs)

                        print('{}:: No {} avCV perfs {}'.format(s,chn,perfs_str))
                        perf_drop = perfs_full - perfs

                        perf_drops[s][chn] = perf_drop
                        print('  Perf drop: ', utsne.sprintfPerfs(perf_drop) )
                    if len(perf_drops[s]) == 0:
                        del perf_drops[s]
                    #print(lda_anver.keys())
    return perf_drops

In [ ]:
winnder_chans = {}
for s,pdcurd in perf_drops.items():
    pds = []
    # I want ordered access across 
    for chn in chnames_LFP:
        pds += [pdcurd[chn]]
    pds = np.vstack(pds)
    # axis 0 -- index of channel
    maxdrop_perchan = np.max(pds,axis=1)
    maxdrop_perchan = np.maximum(maxdrop_perchan,0)
    
    mindrop_perchan = np.min(pds,axis=1)
    mindrop_perchan = np.minimum(mindrop_perchan,0)
    # dropping channel should maximum worsen and minimum improve (mindrop is neg)
    inds = np.argsort(maxdrop_perchan + mindrop_perchan)
    win_ind = inds[-1]
    winnder_chans[s] = chnames_LFP[win_ind]
    #print(pds*100)
    print('{:50} {} {}'.format( s, chnames_LFP[ win_ind ], utsne.sprintfPerfs( pds[win_ind] ) ) )

# Best features plot

In [ ]:
lda_output_pg_cur.keys()

In [ ]:
lda_output.keys()

In [ ]:
lda_output_pg_cur.keys()

In [ ]:
%matplotlib inline

In [ ]:
rncur = rawnames[0]
#prefix = 'all'
#prefix = 'LFPrel_noself'
#prefix = 'allb_beta'
#prefix = 'modSrc'
prefix = 'LFPrel_noself'
prefix = 'all'
prefix = 'modSrc_self'
it_grouping = 'merge_nothing'
it_set = 'basic'
nbest = 50
#nbest_MI = nbest
slice_best_feats = slice(-nbest,None)

lda_output_pg_cur = output_per_raw[rncur][prefix]
lda_output = lda_output_pg_cur[it_grouping][it_set]
featnames_filtered = lda_output_pg_cur['feature_names_filtered']

sind_str,mc,tk = utils.getParamsFromRawname(rncur)


In [ ]:
bfts = ['LDA', 'MI', 'XGB']
bft = bfts[1]

In [ ]:
sind_str,mc,tk  = utils.getParamsFromRawname(rncur)
sources_type='parcel_aal'
src_file_grouping_ind = 10
src_rec_info_fn = '{}_{}_grp{}_src_rec_info'.format(rncur,
                                                    sources_type,src_file_grouping_ind)
src_rec_info_fn_full = os.path.join(gv.data_dir, src_rec_info_fn + '.npz')
rec_info = np.load(src_rec_info_fn_full, allow_pickle=True)


print( list(rec_info.keys()) )

labels_dict = rec_info['label_groups_dict'][()]
srcgroups_dict = rec_info['srcgroups_dict'][()]
coords = rec_info['coords_Jan_actual'][()]
srcgrouping_names_sorted = rec_info['srcgroups_key_order'][()]
sgdn = 'all_raw'

In [ ]:
best_feats_dict = {}
#feat_info_pri
featnames_nice = utils.nicenFeatNames(featnames_filtered,
                                    labels_dict,srcgrouping_names_sorted)
featnames_nice = np.array(featnames_nice)


if 'MI_per_feati' in lda_output:
    MI = lda_output['MI_per_feati']

    strong_inds = np.argsort(MI)
    # increasing oder
    best_feats_dict['MI'] = strong_inds

if 'strong_inds_LDA' in lda_output:
    # increasing oder
    strong_inds = lda_output['strong_inds_LDA']
    best_feats_dict['LDA'] = strong_inds
    
# increasing oder
strong_inds = lda_output['strong_inds_XGB']
best_feats_dict['XGB'] = strong_inds

# increasing oder
strong_inds = lda_output['inds_important']
best_feats_dict['LDA_naive'] = strong_inds

# pca_derived_featinds_perthr = lda_output['pca_derived_featinds_perthr']
# feat_variance_q_thr = lda_output['feat_variance_q_thr']

# for thri,thr in feat_variance_q_thr:
#     best_feats_dict['PCA_{:.2f}'.format(thr)] = pca_derived_featinds_perthr[thri] 

In [ ]:
#bft = 'XGB'
strong_inds = best_feats_dict[bft][slice_best_feats]
#feat_names_cur = featnames_nice[strong_inds]
feat_names_cur = featnames_filtered[strong_inds]


In [ ]:
# print('{}_{}:_{}_{}_{}__{}best descending order\n'.format(sind_str,prefix,
#                                               it_grouping,it_set,bft,nbest))
# feat_names_cur_nice = featnames_nice[strong_inds]
# for fn in feat_names_cur_nice[::-1]:
#     print(fn)

In [ ]:
print('{}_{}:_{}_{}_{}__{}best descending order\n'.format(sind_str,prefix,
                                              it_grouping,it_set,bft,nbest))
feat_names_cur_nice = featnames_nice[strong_inds]
for fn in feat_names_cur_nice[::-1]:
    print(fn)

In [ ]:
import re
src_chns_per_feat_nice = []
src_chns_all_nice = []
parcel_indices = []
parcel_indices_all = []
for fni,fn in enumerate(feat_names_cur):
    p = 'msrc._[0-9]+_[0-9]+_c[0-9]+'
    source_chns_cur_featname = re.findall(p, fn)
    if len(source_chns_cur_featname) == 0:
        continue
    
    sides,groupis,parcelis,compis = utils.parseMEGsrcChnamesShortList(source_chns_cur_featname)
    parcel_indices_all += parcelis
    parcel_indices += [parcelis]
    
    tmp = list(srcgrouping_names_sorted) * 10  # because we have 9 there
    nice_chns_cur_featname = utils.nicenMEGsrc_chnames(source_chns_cur_featname,labels_dict,tmp,
                            prefix='msrc_')
    nice_chns_cur_featname = list(set(nice_chns_cur_featname))
    src_chns_per_feat_nice += [nice_chns_cur_featname]
    src_chns_all_nice += nice_chns_cur_featname
    print(fn,nice_chns_cur_featname)

sgdn='all_raw'
parcel_indices_all = list(set(parcel_indices_all))
#parcel_indices_all

roi_labels = np.array(  labels_dict[sgdn] )
rls = ['unlabeled'] + list( roi_labels[parcel_indices_all] )

srcgrp = np.zeros( srcgroups_dict[sgdn].shape, dtype=srcgroups_dict[sgdn].dtype)

for pii,pi in enumerate(parcel_indices_all):
    srcgrp[srcgroups_dict[sgdn] == pi] = pii + 1 #list(roi_labels).index( rls[pii])


In [ ]:
clrs =  utils.vizGroup2(sind_str,coords,rls,srcgrp, show=False, alpha=.1,
                       figsize_mult=1.5,msz=30, printLog=0)

plt.tight_layout()

figname = '{}_{}:_{}_{}_{}_srcmap_{}best'.format(sind_str,prefix,
                                              it_grouping,it_set,bft,nbest)
plt.gcf().axes[0].set_title(figname)
plt.savefig(figname+'.png')
plt.close()

# Shap

In [ ]:
output_per_raw['S01']

In [ ]:
output_per_raw

In [ ]:
rncur = rawnames[0]
#prefix = 'all'
#prefix = 'LFPrel_noself'
#prefix = 'allb_beta'
#prefix = 'modSrc'
#prefix = 'LFPrel_noself'
prefix = 'modLFP'
it_grouping = 'merge_nothing'
it_set = 'basic'
nbest = 10
#nbest_MI = nbest
slice_best_feats = slice(-nbest,None)

lda_output_pg_cur = output_per_raw[rncur][prefix]
lda_output = lda_output_pg_cur[it_grouping][it_set]
featnames_filtered = lda_output_pg_cur['feature_names_filtered']

In [ ]:
lda_anver = lda_output['lda_analysis_versions']
print(lda_anver.keys())
featset_name = 'all_present_features'
model = lda_anver[featset_name]['CV_aver']['ldaobj']
perfs = lda_anver[featset_name]['CV_aver']['perfs']
utsne.sprintfPerfs(perfs)

In [ ]:
import shap

In [ ]:
X = Ximp_per_raw[rncur][prefix]
gi = gis_per_raw[rncur][prefix]
X_to_fit = X[gi]
print(X_to_fit.shape)
nsamples = 30
Xsubset = shap.utils.sample(X_to_fit, nsamples)

In [ ]:
21 * 20 * 

In [ ]:
# compute SHAP values
explainer= shap.Explainer(model.predict, Xsubset,feature_names=featnames_filtered)
shap_values = explainer(X)

In [ ]:
shap.plots.beeswarm(shap_values, max_display=14)

In [ ]:
shap.plots.scatter(shap_values[:,1], color=shap_values)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
sample_ind = 3
shap.plots.waterfall(shap_values[sample_ind]) #, max_display=14)

# Resort output

## tSNE

In [ ]:
newdir = os.path.join(gv.dir_fig, 'tSNE-like' )
if not os.path.exists(newdir):
    os.mkdir( newdir )
newdir_outer = newdir

import shutil
subskip_val = 1
dirname_suffix = 'subskip{}'.format(subskip_val)

for rawname_ in rawnames:
    subj,medcond,task = utils.getParamsFromRawname(rawname_)
    for prefix in prefixes:
    #    rawname_ = 'S04_off_move'


        #prefix = 'all'
        regex = '{}_{}.*{}_tSNE_.*_subskip{}.pdf'.format(subj,medcond,prefix,subskip_val)
        fnfound = utsne.findByPrefix(gv.dir_fig, None, prefix, regex=regex)
        print(regex,len(fnfound))
        if len(fnfound):
            newdir = os.path.join(newdir_outer, '{}_{}_{}'.format(subj, medcond, dirname_suffix) )

            if not os.path.exists(newdir):
                os.mkdir( newdir )

            for fn in fnfound:
                fnfull_old = os.path.join(gv.dir_fig,fn)
                fnfull = os.path.join(newdir,fn)
                shutil.move(fnfull_old,fnfull)

#    fnfound

## PCA

In [ ]:
newdir = os.path.join(gv.dir_fig, 'PCA-like' )
if not os.path.exists(newdir):
    os.mkdir( newdir )
newdir_outer = newdir

import shutil
dirname_suffix = ''

for rawname_ in rawnames:
    subj,medcond,task = utils.getParamsFromRawname(rawname_)
    for prefix in prefixes:
    #    rawname_ = 'S04_off_move'


        #prefix = 'all'
        regex = '{}_{}.*{}_PCA.*.pdf'.format(subj,medcond,prefix)
        fnfound = utsne.findByPrefix(gv.dir_fig, None, prefix, regex=regex)
        print(regex,len(fnfound))
        if len(fnfound):
            newdir = os.path.join(newdir_outer, '{}_{}_{}'.format(subj, medcond, dirname_suffix) )

            if not os.path.exists(newdir):
                os.mkdir( newdir )

            for fn in fnfound:
                fnfull_old = os.path.join(gv.dir_fig,fn)
                fnfull = os.path.join(newdir,fn)
                shutil.move(fnfull_old,fnfull)

#    fnfound

# Make a pic
## Load table

In [ ]:
tfns = []
tfns += [('XGB','pptable_parcel_aal__nr6_nprefixes18_nlts5_perfs_XGB_CV1.csv')]
tfns += [('LDA','pptable_parcel_aal__nr6_nprefixes18_nlts5_all_present_features_CV1.csv')]

#tfns += [('first',"pred_power_table.csv")]
#tfns += [('Hisrch','pred_power_table_nprefixes4_nlts3_perfs_XGB_CV1_Hisrch.csv')]
#tfns += [('full','pptable_parcel_aal_nprefixes9_nlts1_perfs_XGB_CV1_red1.csv')]
#tfns += ['pred_power_table_nprefixes4_nlts3_perfs_XGB_CV1_red1.csv']
#tfns += ['pred_power_table_nprefixes4_nlts3_perfs_XGB_CV1_red0.csv']

#tfns += ['pptable_parcel_aal_nprefixes6_nlts1_perfs_XGB_CV1_red1.csv']
#tfns += ['pptable_parcel_aal_nprefixes6_nlts1_perfs_XGB_CV1_red0.csv']

#tfns += ['pptable_parcel_aal_nprefixes9_nlts1_perfs_XGB_CV1_red0.csv']
# table_fnames_dict = {'red':table_fname1, 'full':table_fname2, 'Hirsch':table_fname3,
#                     'full2':table_fname4, 'red2':table_fname5, 'full3':table_fname6,
#                     'red3':table_fname7}
#table_labels = list(sorted(table_fnames_dict.keys() ))
#table_labels = list((table_fnames_dict.keys() ))

table_labels = [tfn[0] for tfn in tfns]
#table_labels = ['Hirsch']

In [ ]:
#import matp

In [ ]:
table_labels

In [ ]:


#df['0'][0],df['0'][1]

In [ ]:
table_fname = tfns[0][1]

table_fname_full = os.path.join(gv.dir_fig, table_fname)
print(table_fname_full)
        
df = pd.read_csv(table_fname_full)
num_featsets = len(df.keys())
#for tfn in table_labels:
print(num_featsets)

In [ ]:
df

In [ ]:
np.random.seed(2)

In [ ]:
#seq = np.linspace(0,1,num_featsets-2)
seq = np.random.permutation(num_featsets).astype(float); seq /= np.max(seq)
cmap = plt.cm.gist_rainbow
#cmap = plt.cm.hsv
cyc = plt.cycler("color", cmap(seq ))

plt.cm.gist_rainbow

# for ci,c in enumerate(cyc):
#     print(c)

In [ ]:
des_label = 'allsep'
des_label = 'medcond'
des_featsets = ['all', 'allb_beta', 'modLFP', 'modSrc_self', 'modSrc', 'LFPrel_noself']
featset_dispname = ['MEG+LFP', 'MEG beta+LFP beta', 'LFP', 'MEG', 'MEG_conn', 'cross LFP-MEG']

des_featsets = ['all', 'allb_beta', 'modLFP', 'modSrc_self', 'LFPrel_noself']
featset_dispname = ['MEG+LFP', 'MEG beta+LFP beta', 'LFP', 'MEG', 'cross LFP-MEG']

des_label_disp_name = {'allsep':'tremor', 'medcond':'ON/OFF'}
dlb = des_label_disp_name[des_label]

table_dispname = ['Entire hemisphere', '4 sources', 'Entire hemisphere2']
table_dispname = ['XGB {}'.format(dlb), 'LDA {}'.format(dlb)]

In [ ]:
ww = 7
hh = ww
nr = len(table_labels)
nc = 2
fig,axs = plt.subplots(nr,nc, figsize=(ww*2,hh*nr))
axs = axs.reshape((nr,nc))
defsz = 20; larger= ['all', 'modLFP', 'modSrc', 'allb_beta']
defsz = 120; larger = [];
legend_alpha = 0.3
legend_fontsz=17
title_fontsz=25
title2_fontsz=15

for ax in axs.flatten():
    ax.set_xlim(-3,103)
    ax.set_ylim(-3,103)
    ax.set_ylim(50,103)
    ax.set_xlabel('Sensitivity',fontsize=22)
    ax.set_ylabel('Specificity',fontsize=22)

for rowi in range(nr):
    table_lbl_cur = table_labels[rowi]
    #tfn = table_fnames_dict[table_lbl_cur]
    tfn = tfns[rowi][1]
    tfn_full = os.path.join(gv.dir_fig, tfn)
    df = pd.read_csv(tfn_full)
    
    ax  = axs[rowi,0]
    ax1 = axs[rowi,1]

    num_featsets_cur = len(df.keys())
    featsetcolori = 0
    for ind in range(num_featsets_cur): #range(1,num_featsets-1):
    #ind = 1
        indd = 0
        try:
            indd = int(df.keys()[ind])  #column
        except (ValueError,KeyError) as e:
            indd = 0
        
        if indd <= 0:
            continue
        cc  = list(cyc)[featsetcolori]
        key = str(indd)
        featset_name = df[key][0]
        if featset_name not in des_featsets:
            continue
            
        featsetcolori += 1
        print('{}: featset name {}'.format(table_lbl_cur,featset_name) )
        
        l = len(df[key])
        l = 0
        nice_inds = []
        for i in range(1,len(df[key])):
            if df['0'][i].find(des_label) >= 0:
                #print(df['0'][i])
                l += 1
                nice_inds += [i]

        # specs over patients
#         senss = (l-1)*[0]
#         specs = (l-1)*[0]
        senss = (l)*[np.nan]
        specs = (l)*[np.nan]
        #print(len(specs),nice_inds)
        #for i in range(1,l):
        for ii,i in enumerate(nice_inds):
            s = df[key][i]
            try:
                r = list(map(int,s.split(',')) )
            except ValueError as ve:
                continue
            se = r[0]
            sp = r[1]
            if len(r) > 2:
                f1 =  r[2]
            #print(i)
            senss[ii] = se
            specs[ii] = sp
            #print( se,sp  )
        #print('fd0')
                
        clrs = len(specs) *  [ cc['color'] ]
        sz = defsz
        mrk = 'o'
        if featset_name in larger:
            sz *= 8        
            mrk = 'o'
        ase = np.mean(np.array(senss)[~np.isnan(senss) ])
        asp = np.mean(np.array(specs)[~np.isnan(specs) ])
        lbl = '{} ~ {:.0f},{:.0f}'.format(featset_name, ase, asp)
        lbl = featset_dispname[des_featsets.index(featset_name)]
        
        #print('fd1')
        ax.scatter(senss,specs, c=clrs , label=lbl, s= sz, marker=mrk, alpha=0.8)

        ax1.scatter([ase],[asp], c=[clrs[0]] , label=lbl, s= sz, marker=mrk)

    ax.legend(loc='lower left',fontsize=legend_fontsz,framealpha=legend_alpha); 
    ttl = table_lbl_cur
    ttl = table_dispname[rowi]
    ax.set_title('{} across 6 patients'.format(ttl),
                                              fontsize=title_fontsz)
    ax1.legend(loc='lower left',framealpha=legend_alpha); 
    ax1.set_title('{} Average predictive powers'.format(table_lbl_cur),
                                                fontsize=title2_fontsz)
    ax.grid(1)
    ax1.grid(1)

#plt.tight_layout()
plt.savefig('pred_power_scatter_o{}_{}.pdf'.format(table_fname,des_label))

In [ ]:
len(rawnames)

In [ ]:
df

# Some older stuff

In [ ]:
import matplotlib.pyplot as plt
plt.close('all')

In [ ]:
#%debug
#%run -i run_PCA.py -r S01_off_hold,S01_off_move --pcexpl 0.95 --discard 0.01 --show_plots 0 --prefix all --load_only 1
%run -i run_PCA.py -r S01_off_hold,S01_off_move --pcexpl 0.95 --discard 0.01 --show_plots 0 \
--prefix all --load_only 0

nfeats_per_comp_LDA = 16
r = utsne.getImporantCoordInds(lda.scalings_.T, 
                               nfeats_show = nfeats_per_comp_LDA, q=0.8, printLog = 1)
inds_toshow, strong_inds_pc, strongest_inds_pc  = r


for dim,feat_inds in enumerate(strong_inds_pc):
    print(dim, np.array(feature_names_pri[dim])[feat_inds] )
    #print(dim,feat_inds)

In [ ]:
#%debug
#%run -i run_PCA.py -r S01_off_hold,S01_off_move --pcexpl 0.95 --discard 0.01 --show_plots 0 --prefix all --load_only 1
%run -i run_PCA.py -r S02_off_hold,S02_off_move --pcexpl 0.95 --discard 0.01 --show_plots 0 \
--prefix all --load_only 0

nfeats_per_comp_LDA = 16
r = utsne.getImporantCoordInds(lda.scalings_.T, 
                               nfeats_show = nfeats_per_comp_LDA, q=0.8, printLog = 1)
inds_toshow, strong_inds_pc, strongest_inds_pc  = r


for dim,feat_inds in enumerate(strong_inds_pc):
    print(dim, np.array(feature_names_pri[dim])[feat_inds] )
    #print(dim,feat_inds)

01_off_hold
0 ['H_mob_LFPR01' 'H_act_LFPR01' 'H_act_msrcR_0' 'H_compl_msrcR_0'
 'H_compl_msrcR_4' 'H_compl_msrcR_2' 'H_act_LFPR12' 'H_compl_LFPR12']
 
1 ['H_compl_LFPR01' 'con_tremor_msrcR_4,msrcR_6' 'H_mob_LFPR01'
 'H_compl_LFPR23' 'H_act_msrcR_6' 'H_compl_msrcR_6' 'H_act_LFPR12'
 'H_compl_LFPR12']

01_on_hold
0 ['H_compl_LFPR12' 'rbcorr_corr_tremor_msrcR_6,tremor_msrcR_6'
 'H_act_msrcR_0' 'con_tremor_msrcR_0,msrcR_0' 'H_mob_msrcR_4'
 'con_tremor_msrcR_2,LFPR01' 'H_compl_msrcR_4' 'H_compl_msrcR_0']
 
1 ['con_high_beta_msrcR_4,LFPR01' 'H_mob_msrcR_6' 'H_compl_msrcR_0'
 'H_act_LFPR23' 'H_mob_LFPR01' 'H_mob_msrcR_4' 'H_compl_msrcR_4'
 'H_mob_LFPR23']
 
 02_off_hold
 0 ['H_mob_msrcR_4' 'con_high_gamma_msrcR_4,msrcR_4'
 'con_tremor_msrcR_0,msrcR_0' 'H_act_LFPR12' 'H_compl_LFPR12'
 'H_compl_LFPR01' 'H_compl_msrcR_4' 'con_tremor_msrcR_2,LFPR01']
 
1 ['H_mob_msrcR_0' 'H_act_LFPR01' 'H_compl_LFPR01'
 'con_tremor_msrcR_2,LFPR01' 'con_tremor_msrcR_4,msrcR_6'
 'rbcorr_corr_high_gamma_msrcR_6,high_gamma_msrcR_6'
 'rbcorr_corr_high_gamma_LFPR12,high_gamma_msrcR_6' 'H_compl_LFPR23']
 
 02_on_hold
 0 ['rbcorr_corr_low_gamma_msrcR_6,low_gamma_msrcR_6' 'H_compl_LFPR01'
 'con_tremor_msrcR_0,LFPR12' 'con_high_beta_msrcR_2,msrcR_6'
 'H_act_LFPR23' 'rbcorr_corr_tremor_msrcR_6,tremor_msrcR_6'
 'con_tremor_msrcR_2,LFPR01' 'H_act_LFPR01']
 
1 ['con_tremor_msrcR_2,LFPR23' 'H_act_LFPR12'
 'rbcorr_corr_tremor_msrcR_6,tremor_msrcR_6'
 'rbcorr_corr_low_gamma_LFPR12,low_gamma_msrcR_6'
 'con_tremor_msrcR_4,msrcR_6' 'H_compl_LFPR23' 'H_compl_LFPR12'
 'H_compl_LFPR01']

In [ ]:
feature_names_pri[0]

In [ ]:
#%debug
utsne.plotPCA(pcapts,pca, nPCAcomponents_to_plot,feature_names_all, colors, markers,
            mrk, mrknames, color_per_int_type, tasks,
            pdf=None,neutcolor=neutcolor)


In [ ]:
pcapts.shape

In [ ]:
X_LDA.shape

In [ ]:
# fi = feat_file_pri[0]['feat_info'][()]
# nedgeBins = fi['nedgeBins']

In [ ]:
lda.scalings_.shape

In [ ]:
r = utsne.getImporantCoordInds(lda.scalings_.T, nfeats_show = 140, q=0.8, printLog = 1)
inds_toshow, strong_inds_pc, strongest_inds_pc  = r

In [ ]:
len(inds_toshow)

In [ ]:
feature_names_pri

In [ ]:
rawnames = rawnames_for_PCA

In [ ]:
rn_str = ','.join(rawnames)
fn = 'rbcorr'
#fn = 'bpcorr'
fn = 'H_.*'
Xfeat_cur, names = utsne.selFeatsRegex(Xconcat.T, feature_names_pri[0], ['{}.*'.format(fn)])

In [ ]:
utsne.plotBasicStatsMultiCh(Xfeat_cur, names, printMeans = True, singleAx = 1, shift_std_mult = 6)
plt.savefig('{}_{}_feat_stats.pdf'.format(rn_str,fn))
plt.close()